
<br>
===================================<br>
Masked Normalized Cross-Correlation<br>
===================================<br>
In this example, we use the masked normalized cross-correlation to<br>
identify the relative shift between two similar images containing<br>
invalid data.<br>
In this case, the images cannot simply be masked before computing the<br>
cross-correlation, as the masks will influence the computation. The<br>
influence of the masks must be removed from the cross-correlation, as<br>
is described in [1]_.<br>
In this example, we register the translation between two<br>
images. However, one of the images has about 25% of the pixels which<br>
are corrupted.<br>
.. [1] D. Padfield, "Masked object registration in the Fourier domain"<br>
       IEEE Transactions on Image Processing (2012).<br>
       :DOI:`10.1109/TIP.2011.2181402`<br>


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from skimage import data, draw
from skimage.registration import phase_cross_correlation
from scipy import ndimage as ndi

###########################################<br>
Define areas of the image which are invalid.<br>
Probability of an invalid pixel is 25%.<br>
This could be due to a faulty detector, or edges that<br>
are not affected by translation (e.g. moving object in a window).<br>
See reference paper for more examples

In [ ]:
image = data.camera()
shift = (-22, 13)

In [ ]:
corrupted_pixels = np.random.choice([False, True], size=image.shape,
                                    p=[0.25, 0.75])

The shift corresponds to the pixel offset relative to the reference image

In [ ]:
offset_image = ndi.shift(image, shift)
offset_image *= corrupted_pixels
print(f"Known offset (row, col): {shift}")

Determine what the mask is based on which pixels are invalid<br>
In this case, we know what the mask should be since we corrupted<br>
the pixels ourselves

In [ ]:
mask = corrupted_pixels

In [ ]:
detected_shift = phase_cross_correlation(image, offset_image,
                                         reference_mask=mask)

In [ ]:
print(f"Detected pixel offset (row, col): {-detected_shift}")

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, sharex=True, sharey=True,
                                    figsize=(8, 3))

In [ ]:
ax1.imshow(image, cmap='gray')
ax1.set_axis_off()
ax1.set_title('Reference image')

In [ ]:
ax2.imshow(offset_image.real, cmap='gray')
ax2.set_axis_off()
ax2.set_title('Corrupted, offset image')

In [ ]:
ax3.imshow(mask, cmap='gray')
ax3.set_axis_off()
ax3.set_title('Masked pixels')

In [ ]:
plt.show()

################################################<br>
Solid masks are another illustrating example. In this case, we have<br>
a limited view of an image and an offset image. The masks for these<br>
images need not be the same. The `phase_cross_correlation`<br>
function will correctly identify which part of the images should be<br>
compared.

In [ ]:
image = data.camera()
shift = (-22, 13)

In [ ]:
rr1, cc1 = draw.ellipse(259, 248, r_radius=125, c_radius=100,
                        shape=image.shape)

In [ ]:
rr2, cc2 = draw.ellipse(300, 200, r_radius=110, c_radius=180,
                        shape=image.shape)

In [ ]:
mask1 = np.zeros_like(image, dtype=bool)
mask2 = np.zeros_like(image, dtype=bool)
mask1[rr1, cc1] = True
mask2[rr2, cc2] = True

In [ ]:
offset_image = ndi.shift(image, shift)
image *= mask1
offset_image *= mask2

In [ ]:
print(f"Known offset (row, col): {shift}")

In [ ]:
detected_shift = phase_cross_correlation(image, offset_image,
                                         reference_mask=mask1,
                                         moving_mask=mask2)

In [ ]:
print(f"Detected pixel offset (row, col): {-detected_shift}")

In [ ]:
fig = plt.figure(figsize=(8,3))
ax1 = plt.subplot(1, 2, 1)
ax2 = plt.subplot(1, 2, 2, sharex=ax1, sharey=ax1)

In [ ]:
ax1.imshow(image, cmap='gray')
ax1.set_axis_off()
ax1.set_title('Reference image')

In [ ]:
ax2.imshow(offset_image.real, cmap='gray')
ax2.set_axis_off()
ax2.set_title('Masked, offset image')

In [ ]:
plt.show()